In [38]:
# add bad data - 
#  missing data / rows
# duplicate rows

In [39]:
from pydbgen import pydbgen
import pandas as pd
import random
import numpy
from random import randrange
from datetime import timedelta
from datetime import datetime

In [40]:
myDB = pydbgen.pydb()

In [41]:
customers=myDB.gen_dataframe(
1000, fields=['name','date','city','state','zipcode','phone','email'],
real_email=True,phone_simple=True,real_city=True
)

In [42]:
def apply_gender(x):
    if x <=0.4: gender='male'
    elif x <=0.8: gender='female'
    else: gender='other'
    return gender

In [43]:
customers['gender'] = [random.random() for row in customers['name']]
customers['gender'] = [apply_gender(row) for row in customers['gender']]
customers['customer_id'] = customers.index
customers = customers.rename(columns={'date':'birthdate'})

In [44]:
products = pd.read_csv('products.csv')

In [45]:
food_choices = products['product_id'][products.type =='food'].values
beverage_choices = products['product_id'][products.type =='beverage'].values
extra_choices = products['product_id'][products.type =='extra'].values

In [46]:
def create_order():
    customer_id = random.choice(customers['customer_id'])
    food = []
    drink = []
    extra = []
    complete = False
    cycle_count = 0
    while complete == False:
        cycle_count += 1
        if (0.9 - 0.1*len(food)**2) > random.random():
            food_id = random.choice(food_choices)
            food.append(food_id)
            if 0.9 - 0.2*len(extra)**2 > random.random():
                extra.append(random.choice(products['product_id'][(products['type'] =='extra') & (products['size'] == products['size'][products.product_id == food_id].values[0])].values))
        if (0.25 + 0.5*(len(food)-len(drink)) > random.random()) and ((0.9 - 0.1*len(drink)**2) > random.random()):
            drink.append(random.choice(beverage_choices))
        if (len(food) > 0 or len(drink) > 0) and (.8*cycle_count**2 > random.random()):
            complete = True
    order_items = pd.DataFrame([*food, *drink, *extra], columns=['product_id'])
    order_items['customer_id'] = customer_id
    order_items = order_items.groupby(by=["product_id"]).count().reset_index()
    order_items = order_items.rename(columns ={'customer_id':'quantity'})
    order_items['customer_id'] = customer_id
    order_items = order_items.merge(products[['name','product_id','cost']], on='product_id', how='left')
    orders = order_items
    orders['subtotal'] = orders['quantity']*orders['cost']*100
    orders = orders[['customer_id','quantity','subtotal']].groupby(by='customer_id').sum().reset_index()
    orders['tax'] = int(0.13*orders['subtotal'])
    orders['total'] = orders['subtotal'] + orders['tax']
    orders = orders.astype('int32')
    return order_items, orders

In [47]:
order_items, orders = create_order()
order_items

,product_id,quantity,customer_id,name,cost,subtotal
0,6,1,214,l_fr,4.99,499.0
1,12,1,214,l_ext_grv,0.99,99.0


In [48]:
order_items = pd.DataFrame()
orders = pd.DataFrame()
start = datetime.strptime('1/1/2021 12:00 AM', '%m/%d/%Y %I:%M %p')
for i in range(1,10001):
    item, order = create_order()
    item['order_id'] = i
    order['order_id'] = i
    if i > 1:
        start = orders['created_at'].loc[i-2]
    random_second = randrange(10000)
    created_at = start + timedelta(seconds=random_second)
    item['created_at'] = created_at
    order['created_at'] = created_at
    order_items = pd.concat([order_items, item]).reset_index(drop=True)
    orders = pd.concat([orders, order]).reset_index(drop=True)
    if(random.random() > 0.99):
        orders = pd.concat([orders, order]).reset_index(drop=True)

In [49]:
fulfillment = pd.DataFrame()
for index, row in orders.iterrows():
        order_created = orders[['order_id','customer_id','created_at']].loc[index]
        order_created['event_id'] = 1
        order_created['event_name'] = 'order_created'
        order_id = order_created['order_id']
        customer_id = order_created['customer_id']
        random_second = randrange(10800)
        order_packaged_at = order_created['created_at'] + timedelta(seconds=(random_second+3600))
        order_created = pd.concat([pd.DataFrame(order_created, columns=['order_id','customer_id','created_at','event_id','event_name']), pd.DataFrame([[order_id, customer_id, order_packaged_at, 2, 'order_packaged']], columns=['order_id','customer_id','created_at','event_id','event_name'])])
        random_second = randrange(15800)
        order_shipped_at = order_packaged_at + timedelta(seconds=(random_second+20000))
        order_created = pd.concat([order_created, pd.DataFrame([[order_id, customer_id, order_shipped_at, 3, 'order_shipped']], columns=['order_id','customer_id','created_at','event_id','event_name'])])
        random_second = randrange(30800)
        order_delivered_at = order_packaged_at + timedelta(seconds=(random_second+86400))
        order_created = pd.concat([order_created, pd.DataFrame([[order_id, customer_id, order_delivered_at, 3, 'order_delivered']], columns=['order_id','customer_id','created_at','event_id','event_name'])])
        fulfillment = pd.concat([fulfillment, order_created])

In [50]:
orders.to_csv('orders.csv')
order_items.to_csv('order_items.csv')
fulfillment.to_csv('fulfillment.csv')
customers.to_csv('customers.csv')